# Bug Reference Utils

Code snippets useful for working with bug references and bug trackers

In [ ]:
include("../src/init.jl") # Configure and install (if necessary) the JDP project

import JDP.BugRefs
import JDP.Trackers.Bugzilla

Extract bug references from the specified string and display them as hyper links using Markdown.

In [ ]:
BugRefs.extract_refs("bsc#1111458 bsc#1111464 bsc#1111466 bsc#1111469") |> BugRefs.to_md

In [ ]:
BugRefs.extract_refs("bsc#1110703 bsc#1111488") |> BugRefs.to_md
    

Displaying them as hyperlinks is OK, but we can go further by automatically fetching data from Bugzilla and displaying it here. First we need to login to Bugzilla, if you have not set a user name and password in the trackers config file then you will be prompted for them.

> WARNING: This initially uses HTTP basic auth which inserts your user name and password into the URI, however subsequent requests use a cookie stored in the ses object

In [ ]:
ses = Bugzilla.login("bsc");

Now we can get a bug. The object returned is a Dictionary derived from Bugzilla's XML output.

In [ ]:
bug = Bugzilla.get_bug(ses, 1103543)

This can be converted to a Markdown object which Jupyter will display nicely.

In [ ]:
md = Bugzilla.to_md(bug)

We can take a bunch of bugs from the report notebook and display a summary of all of them.

In [ ]:
bugs = []

for ref in ("bsc#1099134", "bsc#1074293", "poo#35347", "bsc#1099173", "bsc#1102358", "bsc#1102250", "bsc#1108010", "bsc#1108028", "poo#40424")
    if startswith(ref, "bsc")
        push!(bugs, Bugzilla.get_bug(ses, parse(Int, ref[5:end])))
    end
end

bugs

In [ ]:
using Markdown
import Markdown: Paragraph, MD

map(Bugzilla.to_md, bugs) |> MD